In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

# 训练RF训练器

In [ ]:
# 加载样本集
samples = ee.FeatureCollection('users/311605001111/wuhan_new2020')

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

filtered_samples = k_mean(samples)
print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())

# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'
trainedClassifier_filter = ee.Classifier.smileRandomForest(20).train(filtered_samples,label,bands)

# 函数定义

In [5]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

def maskSR(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# Landsat7与Landsat8合并

In [11]:
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate('2020-01-01', '2020-12-31')
           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2020-01-01', '2020-12-31')
landsat_images = l8.merge(l7).map(water_index).map(maskSR)
Map.addLayer(l8.mosaic().clip(roi), visParams,'image mosaic')
print(l7.size().getInfo())
print(l8.size().getInfo())
print(landsat_images.size().getInfo())
# print(l7.first().get('system:id').getInfo())
# print(l8.first().get('system:id').getInfo())
# print(landsat_images.first().get('system:id').getInfo())

51
47
98
LANDSAT/LE07/C01/T1_SR/LE07_122038_20200210
LANDSAT/LC08/C01/T1_SR/LC08_122038_20200218
LANDSAT/LC08/C01/T1_SR/LC08_122038_20200218


# 计算水体频率

## Landsat7

In [ ]:
dataset = l7.map(water_index).map(maskSR)
# 水体的有效个数
waterpixel_count = dataset.map(lambda i : i.select(bands).classify(trainedClassifier_filter).eq(1).rename('waterclass')).sum()
Map.addLayer(waterpixel_count,{'palette':['white','#3dd633'],'min':0,'max':35},"waterPixelCount7")
# 像元的有效个数
validPixel = dataset.count().select('B3').clip(roi).rename('count')
Map.addLayer(validPixel,{'palette':['white','#e40775'],'min':0,'max':35},"validPixel7")
# 水体频率
waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','blue'],'min':0,'max':1},"frequency7")
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
Map.addLayer(permanentwater.selfMask(),{'palette':['blue']},"permanent water7")

## landsat8

In [ ]:
dataset = l8.map(water_index).map(maskSR)
# 水体的有效个数
waterpixel_count = dataset.map(lambda i : i.select(bands).classify(trainedClassifier_filter).eq(1).rename('waterclass')).sum()
Map.addLayer(waterpixel_count,{'palette':['white','#3dd633'],'min':0,'max':35},"waterPixelCount8")
# 像元的有效个数
validPixel = dataset.count().select('B3').clip(roi).rename('count')
Map.addLayer(validPixel,{'palette':['white','#e40775'],'min':0,'max':35},"validPixel8")
# 水体频率
waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','blue'],'min':0,'max':1},"frequency8")
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
Map.addLayer(permanentwater.selfMask(),{'palette':['blue']},"permanent water8")

## landsat7+landsat8

In [ ]:
dataset = landsat_images
# 水体的有效个数
waterpixel_count = dataset.map(lambda i : i.select(bands).classify(trainedClassifier_filter).eq(1).rename('waterclass')).sum()
Map.addLayer(waterpixel_count,{'palette':['white','#3dd633'],'min':0,'max':35},"waterPixelCount78")
# 像元的有效个数
validPixel = dataset.count().select('B3').clip(roi).rename('count')
Map.addLayer(validPixel,{'palette':['white','#e40775'],'min':0,'max':35},"validPixel78")
# 水体频率
waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','blue'],'min':0,'max':1},"frequency78")
permanentwater = waterfrequency.gte(0.75).select('frequency').rename('waterclass')
Map.addLayer(permanentwater.selfMask(),{'palette':['blue']},"permanent water78")

# 制图

In [ ]:
# validPixel,waterpixel_count
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

minmax = waterpixel_count.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

## 水体像元

In [ ]:
# 水体像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
fig = plt.figure(figsize=(12, 8))
visParams = {'min': 0.0,'max': 30.0,'palette': ['FFFFFF','0000FF']}
ax = cartoee.get_map(waterpixel_count,region=region,vis_params=visParams)
cartoee.add_colorbar(ax, vis_params=visParams,loc="right",label="number", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'water pixels in 2020(landsat7+landsat8)', fontsize=20)

## 有效像元

In [ ]:
# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
fig = plt.figure(figsize=(12, 8))
visParams = {'min': 0.0,'max': 48.0,'palette': ['FFFFFF','FF0000']}
ax = cartoee.get_map(validPixel,region=region,vis_params=visParams)
cartoee.add_colorbar(ax, vis_params=visParams,loc="right",label="number", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'valid pixels in 2020(landsat7+landsat8)', fontsize=20)

## 水体频率

In [ ]:
# 水体频率
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency,region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2020(landsat7+landsat8)', fontsize=20)

## 永久性水体

In [ ]:
# 永久性水体
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent water in 2020(landsat8)', fontsize=20)

In [ ]:
# # # 大庆市
# # roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','大庆市'))
# # # 天津市
# # roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','天津市'))
# # 武汉市
# roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
# # # 珠三角
# # roi = ee.Geometry.Rectangle([112.846779, 22.565455,113.682786, 23.114128])
# # # 青藏高原
# # roi = ee.Geometry.Rectangle([86.974365, 30.431624,91.650147, 33.694569])
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,9)

In [ ]:
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate('2020-01-01', '2020-12-31') \
            .filter(ee.Filter.lt('CLOUD_COVER',5)) \
            .map(maskL8sr)
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

print(dataset.size().getInfo())
print(dataset.aggregate_array('system:index').getInfo())

In [ ]:
def cloudMaskL457(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}
dataset = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate('2020-02-01', '2020-03-31') \
            .filter(ee.Filter.lt('CLOUD_COVER',5))

Map.addLayer(dataset.first().clip(roi), visParams,'first')
Map.addLayer(cloudMaskL457(dataset.first()).clip(roi), visParams,'mask first')
# LE07_123038_20200217

In [ ]:
# sentinel-2数据
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)
visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B8', 'B3', 'B2'],
}

# dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
#     .filterBounds(roi) \
#     .filterDate('2020-06-01', '2020-07-31') \
#     .filter(ee.Filter.eq('MGRS_TILE','49RGP')) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
#     .map(maskS2clouds)
# Map.addLayer(dataset.first().clip(roi), visualization, 'RGB')

dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(bound) \
    .filterDate('2020-03-01', '2020-04-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
    .map(maskS2clouds)

print(dataset.size().getInfo())
print(dataset.aggregate_array('system:index').getInfo())

In [ ]:
first = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_123038_20200217')
Map.addLayer(first.clip(roi), {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4},'first image ')